# EDA

## intro

#### plan

now just need to:
remove most of the rows that had warnings
extract wiki-variables (features/targets) by category:
daily edit count
daily aggregate article expansion/contraction (+/- characters)
daily # of unique users editing
pick some models to train and compare:
which wiki-vars are targets
which wiki-vars are features
what window-functions for lags (make stationary model)
(do external features later)

#### imports

In [16]:
import pandas as pd, numpy as np, os, re, pyperclip
import mysql.connector as mysql, sqlalchemy
from datetime import datetime as dt

#### get creds, connect to mariadb

In [6]:
mysql_user = 'bhrdwj'
# mysql_pass = input(f'Enter the MySQL password for user {mysql_user}: ')

In [7]:
host='localhost'; user=mysql_user; passwd=mysql_pass; dbname='jawiki';
cxn = mysql.connect(host=host,user=user,passwd=passwd, database=dbname)
cur = cxn.cursor()

connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
try:
    engine = sqlalchemy.create_engine(connection_str)
    conn = engine.connect()
except Exception as e:
    print('Database connection error - check creds')
    print(e)

In [8]:
metadata = sqlalchemy.MetaData(conn)
metadata.reflect()
current_tables = list(metadata.tables.keys())
current_tables

['category',
 'categorylinks',
 'page',
 't2013',
 't2014',
 't2015',
 't2016',
 't2017',
 't2018',
 't2019',
 't2020',
 't2021']

#### some paths or something

In [15]:
datapath = '../data/raw/jawiki/dumps_unzipped/'

## union

### functions ```get_flat_list```, ```jpr_sql```

##### get_flat_list

In [9]:
import itertools as IT
import collections

def flatten(iterable, ltypes=collections.abc.Iterable):
    """This version of flatten avoids python's recursion limit 
      (and thus works with arbitrarily deep, nested iterables).
      It is a generator which can handle strings and arbitrary iterables 
      (even infinite ones).
    By unutbu:
      https://stackoverflow.com/a/2158562/16133813
    """
    
    remainder = iter(iterable)
    while True:
        first = next(remainder)
        if isinstance(first, ltypes) and not isinstance(first, (str, bytes)):
            remainder = IT.chain(first, remainder)
        else:
            yield first

def get_flat_list(iterable):
    gen = flatten(iterable)
    mylist = []
    still_going = True
    while still_going:
        try:
            next_item = next(gen)
            mylist.append(next_item)
        except Exception as e:
            print(e)
            still_going = False
    return mylist

##### jpr_sql

In [10]:
def jpr_sql(query, conn=conn):
    try: 
        a = conn.execute(query)
    except Exception as e:
        print('Printing execute error:')
        print(e)
        return None
    
    try:
        return a.fetchall()
    except Exception as e:
        print('Printing results error:')
        print(e)
        print('Printing result')
        return None

### test union

In [13]:
jpr_sql("""
with hi as 
    ((select * from t2013)
    union all
    (select * from t2014))
select count(*) from hi;
""")

[(9039710,)]

### view

## clean bad rows

### preliminary MVP: drop all rows in warnings

##### get all the mariadb logs for loading history data

In [21]:
warnpath = datapath + 'load_data_warnings/'
logs = sorted(os.listdir(warnpath))

##### make dictionary of lists of unique warnings-rownums, one list per log

In [64]:
warnnums = {}
rownum_regex = re.compile(r'(\d*)(?= *\|\n$)')
for logfile in logs:
    thislog_warnnums = []
    with open(warnpath + logfile) as log:
        for line in log.readlines():
            if re.match(r'^\| Warning \|', line):
                rownum_str = re.search(rownum_regex, line).group()
                rownum_int = int(rownum_str)
                thislog_warnnums.append(rownum_int)
    warnnums[logfile] = list(set(thislog_warnnums))

In [69]:
len(warnnums), {i:len(warnnums[i]) for i in warnnums}

(9,
 {'t2013.log': 65,
  't2014.log': 106,
  't2015.log': 104,
  't2016.log': 106,
  't2017.log': 132,
  't2018.log': 231,
  't2019.log': 339,
  't2020.log': 806,
  't2021.log': 1454})

##### check that the problem rows are the onese that were removed by some spot-checking

In [39]:
some_logs = []
with open(warnpath + logs[0]) as log:
    line = log.readline()
    print(line)
    print(re.match(r'\| Warning \|', line))

MariaDB [jawiki]> show warnings;

None
